import dataset

In [43]:
import pandas as pd 

housing_advert = pd.read_csv('housing-Raoofi.csv')

C:\Users\M7600QE\AppData\Local\Temp\ipykernel_18680\2671681662.py:3: DtypeWarning: Columns (0,1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  housing_advert = pd.read_csv('housing-Raoofi.csv')


data preprocessing

In [44]:
housing_advert.sample(5)

,Area,Construction,Room,Warehouse,Parking,Elevator,Address,Price
86473,140,1396,3,True,True,True,۱ هفته پیش در تهران، دریاچه شهدای خلیج فارس,۸٬۷۰۰٬۰۰۰٬۰۰۰ تومان
82779,145,1401,3,True,True,True,۱ روز پیش در تهران، پاسداران,۲۱٬۳۱۵٬۰۰۰٬۰۰۰ تومان
7260,153,1382,3,True,True,True,۲۳ ساعت پیش در تهران، حکمت,۱۵٬۴۰۰٬۰۰۰٬۰۰۰ تومان
9702,160,1380,3,True,True,True,۱ روز پیش در تهران، شهرک غرب,۱۹٬۲۰۰٬۰۰۰٬۰۰۰ تومان
8155,106,1401,2,True,True,True,۱ روز پیش در تهران، سرتخت,۵٬۶۰۰٬۰۰۰٬۰۰۰ تومان


In [45]:
housing_advert.shape

(94036, 8)

In [46]:
housing_advert.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94036 entries, 0 to 94035
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Area          94036 non-null  object
 1   Construction  94036 non-null  object
 2   Room          94036 non-null  object
 3   Warehouse     94036 non-null  object
 4   Parking       94036 non-null  object
 5   Elevator      94036 non-null  object
 6   Address       94036 non-null  object
 7   Price         94036 non-null  object
dtypes: object(8)
memory usage: 5.7+ MB


In [66]:
df = housing_advert.copy()

In [67]:
df['Area'] = pd.to_numeric(df['Area'], errors='coerce')
df['Construction'] = pd.to_numeric(df['Construction'], errors='coerce')
df['Room'] = pd.to_numeric(df['Room'], errors='coerce')

In [68]:
df['Warehouse'] = df['Warehouse'].astype(bool)
df['Parking'] = df['Parking'].astype(bool)
df['Elevator'] = df['Elevator'].astype(bool)

In [69]:
import re
def clean_address(address):
    match = re.search(r'پیش در تهران، ([^،]+)', address)
    if match:
        return match.group(1)
    return address

In [70]:
df['Address'] =  df['Address'].apply(clean_address)

In [71]:
def clean_price(price):
    cleaned_price = re.sub(r'[^\d]', '', price)
    return int(cleaned_price) if cleaned_price else None

In [72]:
df['Price'] = df['Price'].apply(clean_price)

In [73]:
df.isna().sum()

Area              1
Construction      1
Room              1
Warehouse         0
Parking           0
Elevator          0
Address           0
Price           246
dtype: int64

In [74]:
df.dropna(inplace=True)

In [75]:
df.describe()

,Area,Construction,Room,Price
count,93790.000000,93790.000000,93790.000000,9.379000e+04
mean,125.018083,1392.983463,2.225301,1.452574e+10
std,267.611046,7.745195,0.765839,2.737502e+10
min,1.000000,1371.000000,1.000000,2.300000e+08
25%,74.000000,1386.000000,2.000000,4.900000e+09
50%,105.000000,1395.000000,2.000000,8.580000e+09
75%,141.000000,1401.000000,3.000000,1.550000e+10
max,36500.000000,1401.000000,4.000000,1.000000e+12
